In [4]:
## This file implements neural networks and random forest on p002ypresabs_qual.
## Due to the imbalanced data and the limited cases for class 2, we implement the over-sampling method method.
## For fully-connected neural networks, the accuracy is 96.00% for over-sampling data and 92.5% after regularization.
## For random forest, the accuracy is 95% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 95.49%.

In [5]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [6]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p002ypresabs_qual.csv')
df.shape

(255, 1759)

In [7]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [8]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     1
12     0
13     0
14     0
15     0
16     0
17     0
18     1
19     0
20     0
21     0
22     0
23     0
24     1
25     0
26     0
27     0
28     0
29     0
      ..
225    0
226    0
227    0
228    1
229    1
230    0
231    0
232    0
233    0
234    0
235    0
236    0
237    0
238    0
239    0
240    0
241    0
242    0
243    0
244    0
245    0
246    0
247    0
248    0
249    0
250    0
251    0
252    0
253    0
254    0
Name: pheno, Length: 255, dtype: int64

In [9]:
df['pheno'].value_counts()

0    222
1     30
2      3
Name: pheno, dtype: int64

In [10]:
df.head()

,id,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATTTTCAAATCCCAT,...,group_8208,group_8664,group_8665,group_8666,group_8667,group_8836,group_8913,group_9026,group_9123,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,115,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,1
3,120335,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,120337,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df_clean = df.drop(columns=['id'])

In [12]:
df_clean.shape

(255, 1758)

In [13]:
df_clean.head()

,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATTTTCAAATCCCAT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,...,group_8208,group_8664,group_8665,group_8666,group_8667,group_8836,group_8913,group_9026,group_9123,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [14]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 1757) (255,)


In [15]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 222), (1, 222), (2, 222)]


Using TensorFlow backend.


In [16]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [17]:
############# Fully-Connected Neural Network ################

In [18]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [19]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [20]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 466 samples, validate on 200 samples
Epoch 1/100
466/466 [==============================] - 0s 889us/step - loss: 0.7175 - accuracy: 0.6803 - val_loss: 0.5066 - val_accuracy: 0.8300
Epoch 2/100
466/466 [==============================] - 0s 117us/step - loss: 0.4465 - accuracy: 0.8262 - val_loss: 0.4101 - val_accuracy: 0.8300
Epoch 3/100
466/466 [==============================] - 0s 97us/step - loss: 0.3642 - accuracy: 0.8498 - val_loss: 0.3751 - val_accuracy: 0.8350
Epoch 4/100
466/466 [==============================] - 0s 101us/step - loss: 0.3613 - accuracy: 0.8391 - val_loss: 0.3614 - val_accuracy: 0.8350
Epoch 5/100
466/466 [==============================] - 0s 104us/step - loss: 0.3229 - accuracy: 0.8476 - val_loss: 0.4684 - val_accuracy: 0.7500
Epoch 6/100
466/466 [==============================] - 0s 94us/step - loss: 0.3565 - accuracy: 0.8476 - val_loss: 0.3103 - val_accuracy: 0.8850
Epoch 7/100
466/466 [==============================] - 0s 90us/step - loss: 0.3307 - a

466/466 [==============================] - 0s 151us/step - loss: 0.0335 - accuracy: 0.9957 - val_loss: 0.1186 - val_accuracy: 0.9500
Epoch 58/100
466/466 [==============================] - 0s 171us/step - loss: 0.0288 - accuracy: 0.9957 - val_loss: 0.1195 - val_accuracy: 0.9500
Epoch 59/100
466/466 [==============================] - 0s 169us/step - loss: 0.0328 - accuracy: 0.9936 - val_loss: 0.1538 - val_accuracy: 0.9450
Epoch 60/100
466/466 [==============================] - 0s 139us/step - loss: 0.0311 - accuracy: 0.9957 - val_loss: 0.1187 - val_accuracy: 0.9500
Epoch 61/100
466/466 [==============================] - 0s 136us/step - loss: 0.0292 - accuracy: 0.9979 - val_loss: 0.1391 - val_accuracy: 0.9500
Epoch 62/100
466/466 [==============================] - 0s 121us/step - loss: 0.0260 - accuracy: 0.9979 - val_loss: 0.1661 - val_accuracy: 0.9300
Epoch 63/100
466/466 [==============================] - 0s 126us/step - loss: 0.0354 - accuracy: 0.9936 - val_loss: 0.1284 - val_accuracy

In [22]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

200/200 [==============================] - 0s 67us/step
over-sampling test accuracy: 96.00%


In [23]:
#### add regularizor and dropout
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [24]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 466 samples, validate on 200 samples
Epoch 1/100
466/466 [==============================] - 0s 457us/step - loss: 3.7215 - accuracy: 0.5515 - val_loss: 0.7979 - val_accuracy: 0.7850
Epoch 2/100
466/466 [==============================] - 0s 100us/step - loss: 3.5911 - accuracy: 0.6459 - val_loss: 0.7817 - val_accuracy: 0.7350
Epoch 3/100
466/466 [==============================] - 0s 115us/step - loss: 3.2425 - accuracy: 0.6609 - val_loss: 0.7964 - val_accuracy: 0.7900
Epoch 4/100
466/466 [==============================] - 0s 101us/step - loss: 2.9950 - accuracy: 0.6824 - val_loss: 0.7447 - val_accuracy: 0.7950
Epoch 5/100
466/466 [==============================] - 0s 98us/step - loss: 2.9748 - accuracy: 0.6910 - val_loss: 0.8179 - val_accuracy: 0.8000
Epoch 6/100
466/466 [==============================] - 0s 105us/step - loss: 3.2541 - accuracy: 0.6674 - val_loss: 1.0923 - val_accuracy: 0.8150
Epoch 7/100
466/466 [==============================] - 0s 99us/step - loss: 3.3066 - 

Epoch 57/100
466/466 [==============================] - 0s 104us/step - loss: 1.6958 - accuracy: 0.7682 - val_loss: 0.8830 - val_accuracy: 0.8550
Epoch 58/100
466/466 [==============================] - 0s 95us/step - loss: 1.6603 - accuracy: 0.7361 - val_loss: 0.9479 - val_accuracy: 0.8750
Epoch 59/100
466/466 [==============================] - 0s 88us/step - loss: 1.6454 - accuracy: 0.7339 - val_loss: 0.8854 - val_accuracy: 0.8800
Epoch 60/100
466/466 [==============================] - 0s 87us/step - loss: 1.5003 - accuracy: 0.7682 - val_loss: 0.9883 - val_accuracy: 0.8550
Epoch 61/100
466/466 [==============================] - 0s 84us/step - loss: 1.2994 - accuracy: 0.7918 - val_loss: 0.9621 - val_accuracy: 0.8650
Epoch 62/100
466/466 [==============================] - 0s 86us/step - loss: 1.5027 - accuracy: 0.7339 - val_loss: 0.9302 - val_accuracy: 0.8700
Epoch 63/100
466/466 [==============================] - 0s 97us/step - loss: 1.4317 - accuracy: 0.7597 - val_loss: 0.9733 - val_a

In [26]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

200/200 [==============================] - 0s 113us/step
over-sampling test accuracy: 90.50%


In [27]:
############## Random Forest ##############

In [28]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [29]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 95.00%


In [31]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.96808511 1.         0.94623656 0.93548387 0.92473118]
0.9549073438572409
